In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
IMAGE_SIZE = 64
CHANNELS = 1
BATCH_SIZE = 256
Z_DIM = 100
EPOCHS = 300
LEARNING_RATE = 0.0002
NOISE_PARAM = 0.1

# Prepare the data
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform):
        self.root_dir = root_dir
        self.transform = transform
        self.images = [os.path.join(root_dir, img) for img in os.listdir(root_dir) if img.endswith(".png")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        img = self.transform(img)
        return img


dataset = CustomDataset(root_dir="/kaggle/input/gan-dataset/waqar_pics", transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# Define the generator
class Generator(nn.Module):
    def __init__(self, z_dim, channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(64, channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z.view(z.size(0), z.size(1), 1, 1))

# Define the discriminator
class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(channels, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Flatten(),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize generator and discriminator
generator = Generator(Z_DIM, CHANNELS).to(device)
discriminator = Discriminator(CHANNELS).to(device)

# Loss function and optimizers
criterion = nn.BCELoss()
d_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
g_optimizer = optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

# Training loop
for epoch in range(EPOCHS):
    for real_images in dataloader:
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Train discriminator
        d_optimizer.zero_grad()
        real_labels = torch.ones(batch_size, 1).to(device)
        real_outputs = discriminator(real_images)
        real_loss = criterion(real_outputs, real_labels)
        real_loss.backward()

        z = torch.randn(batch_size, Z_DIM, 1, 1).to(device)
        fake_images = generator(z)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        fake_outputs = discriminator(fake_images.detach())
        fake_loss = criterion(fake_outputs, fake_labels)
        fake_loss.backward()

        d_optimizer.step()

        # Train generator
        g_optimizer.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        g_optimizer.step()

    # Print progress
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{EPOCHS}], d_loss: {real_loss.item() + fake_loss.item()}, g_loss: {g_loss.item()}")

    # Save generated images
    if epoch % 50 == 0:
        folder_name = f"{epoch}_epochs_pics"
        os.makedirs(folder_name, exist_ok=True)

        with torch.no_grad():
            fake_images = generator(torch.randn(25, Z_DIM, 1, 1).to(device)).detach().cpu()

        for i in range(fake_images.size(0)):
            vutils.save_image(fake_images[i], f"{folder_name}/generated_image_epoch_{epoch}_{i}.png", normalize=True)


# Save models
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")


Epoch [0/300], d_loss: 0.3186000809073448, g_loss: 6.252570152282715
Epoch [10/300], d_loss: 0.8392009735107422, g_loss: 2.5531067848205566
Epoch [20/300], d_loss: 0.8546193018555641, g_loss: 1.1791412830352783
Epoch [30/300], d_loss: 0.5591198392212391, g_loss: 3.754275321960449
Epoch [40/300], d_loss: 0.3303839787840843, g_loss: 3.7111949920654297
Epoch [50/300], d_loss: 0.32125783897936344, g_loss: 2.2335636615753174
Epoch [60/300], d_loss: 0.1438494697213173, g_loss: 3.759979248046875
Epoch [70/300], d_loss: 0.19290368258953094, g_loss: 4.202148914337158
Epoch [80/300], d_loss: 0.15114159882068634, g_loss: 4.324741363525391
Epoch [90/300], d_loss: 0.1481534205377102, g_loss: 4.21136999130249
Epoch [100/300], d_loss: 0.14825372397899628, g_loss: 3.4284093379974365
Epoch [110/300], d_loss: 0.09734949469566345, g_loss: 4.145509719848633
Epoch [120/300], d_loss: 0.2027099598199129, g_loss: 3.428286552429199
Epoch [130/300], d_loss: 0.16073133051395416, g_loss: 3.5599851608276367
Epoch 

In [ ]:
import shutil
import os

# Zip and download folders
for epoch in range(0, EPOCHS + 1, 50):
    folder_name = f"{epoch}_epochs_pics"
    shutil.make_archive(folder_name, 'zip', folder_name)

# Move zip files to /kaggle/working directory
for epoch in range(0, EPOCHS + 1, 50):
    folder_name = f"{epoch}_epochs_pics"
    shutil.move(f"{folder_name}.zip", f"/kaggle/working/{folder_name}.zip")

# Create a zip file containing all the zip files
shutil.make_archive("all_epochs_pics", 'zip', "/kaggle/working")
